Code reference: 
+ http://ivanjo39191.pixnet.net/blog/post/81852030-python-爬蟲練習紀錄2%28二%29
+ https://blog.gtwang.org/programming/python-beautiful-soup-module-scrape-web-pages-tutorial/

In [1]:
from urllib import request
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from urllib.request import urlopen
from urllib.error import HTTPError
import re
import sqlite3
from selenium import webdriver
import json
import requests

In [2]:
import pandas as pd
movie_data = pd.read_csv('tmdb_5000_movies.csv')

In [3]:
movie_title = movie_data.original_title.values

In [4]:
movie_title

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

In [5]:
target_url = "http://www.boxofficemojo.com/search/?q="
chromedriver = "/Users/steve/Desktop/chromedriver"
home_url = "http://www.boxofficemojo.com"

movie_link = []
find_fail_list = []

In [37]:
curr_index = len(movie_link)+len(find_fail_list)
for title_index in range(curr_index, len(movie_title)):
    title = movie_title[title_index]
    driver = webdriver.Chrome(chromedriver)
    driver.get(target_url+title)
    ps = driver.page_source

    sp = BeautifulSoup(ps, 'html.parser')
    search_result = sp.find_all("a", href=re.compile("\/movies\/\?id=.*"))
    
    if len(search_result) > 1:
        movie_link.append(home_url + search_result[-1]['href'])
        
    else:
        driver.close()
        tmp = title.split(' ')
        initialism = ''
        l = len(tmp)
        for i in range(l):
            s = tmp[i]
            if i == l-1 and s.isdigit():
                initialism += " "
            initialism += s[0]
                
        driver = webdriver.Chrome(chromedriver)
        driver.get(target_url+initialism)
        ps = driver.page_source

        sp = BeautifulSoup(ps, 'html.parser')
        search_result = sp.find_all("a", href=re.compile("\/movies\/\?id=.*"))
        if len(search_result) > 1:
            movie_link.append(home_url + search_result[-1]['href'])
        else:
            find_fail_list.append(title)

    driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=66.0.3359.139)
  (Driver info: chromedriver=2.33.506106 (8a06c39c4582fbfbab6966dbb1c38a9173bfb1a2),platform=Mac OS X 10.12.6 x86_64)


In [56]:
len(movie_link)

3308

In [57]:
len(find_fail_list)

255

In [40]:
find_fail_list[-1]

'No se Aceptan Devoluciones'

In [54]:
title

'Paranormal Activity 4'

In [55]:
movie_title[len(movie_link)+len(find_fail_list)]

'Paranormal Activity 4'

In [58]:
with open('movie_link.txt', 'w') as f:
    for link in movie_link:
        f.write(link+',')
with open('movie_link_fail.txt', 'w') as f:
    for link in find_fail_list:
        f.write(link+'$')    

In [133]:
weekend_url = movie_link[200].split('?')
weekend_url = weekend_url[0] + '?page=weekend&' + weekend_url[1]
driver = webdriver.Chrome(chromedriver)
driver.get(weekend_url)
ps = driver.page_source

sp = BeautifulSoup(ps, 'html.parser')
#search_result = sp.find_all("a", href=re.compile("\/movies\/\?id=.*"))

trs = sp.find_all("font", {'face':["Verdana"]})
driver.close()
trs[1].string

In [116]:
MOVIE_WEEKEND = []
MOVIE_LIST = []

In [68]:
trs = sp.find_all("tr", bgcolor=["#ffffff","#f4f4ff"])
for i in range(len(MOVIE_LIST),len(movie_link)):
    #get url
    link = movie_link[i]
    weekend_url = link.split('?')
    weekend_url = weekend_url[0] + '?page=weekend&' + weekend_url[1]
    #activate webdriver
    driver = webdriver.Chrome(chromedriver)
    driver.get(weekend_url)
    ps = driver.page_source
    sp = BeautifulSoup(ps, 'html.parser')
    
    trs = sp.find_all("font", {'face':["Verdana"]})
    MOVIE_LIST.append(trs[1].string)
    
    trs = sp.find_all("tr", bgcolor=["#ffffff","#f4f4ff"])
    for tr in trs:
        fts = tr.find_all("font",size="2")
        if len(fts)==9:
            gross = fts[2].string
            MOVIE_WEEKEND.append(gross.replace('$','').replace(',',''))
            break

AttributeError: ResultSet object has no attribute 'text'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [ ]:
'http://www.boxofficemojo.com/movies/?'+'page=weekend'+'&id=avatar.htm'